In [1]:
# Reload all modules every time before executing the Python code typed
%load_ext autoreload
%autoreload 2

In [392]:
import os
import datetime
import re
import json
import cProfile
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import ray
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon, box
import geopandas as geopd
import pandas as pd
import querier as qr
from dotenv import load_dotenv
load_dotenv()

import ses_ling.utils.geometry as geo_utils
import ses_ling.data.access as data_access
import ses_ling.data.filter as data_filter
import ses_ling.data.user_residence as user_residence
import ses_ling.utils.paths as path_utils

In [ ]:
import logging
LOGGER = logging.getLogger(__name__)
LOG_CONFIG = json.load(open('logging.json', 'rt'))
logging.config.dictConfig(LOG_CONFIG)

In [452]:
paths = path_utils.ProjectPaths()

In [5]:
with open(paths.ext_data / 'countries.json') as f:
    countries_dict = json.load(f)

In [591]:
cc = 'GB'
cc_dict = countries_dict[cc]
xy_proj = cc_dict['xy_proj']
year_from = 2015
year_to = 2021
colls = cc_dict['mongo_coll']
timezone = cc_dict['timezone']
paths.partial_format(cc=cc, year_from=year_from, year_to=year_to, **cc_dict)

# Deprivation index

In [5]:
from libpysal.weights import Rook
from spopt.region import WardSpatial
from sklearn.cluster import AgglomerativeClustering

## Wales

THese are ranks! So averaging or inter-lsoa comparisons make no sense!

In [ ]:
zip_fpaths = list((paths.ext_data / 'wimd2014').glob('*.zip'))
# put overall first because different format, simply more practical and more logical
zip_fpaths.sort(key=lambda p: 'overall' not in p.stem)
wimd = geopd.GeoDataFrame.from_file(zip_fpaths[0]).set_index('2014_lsoa')
for zipf in zip_fpaths[1:]:
    part = geopd.GeoDataFrame.from_file(zipf).set_index('2014_lsoa')
    part_var = zipf.stem.split('_')[-1]
    var_col_mask = part.columns.str.startswith('2014_')# | part.columns.str.endswith('_2014')
    var_col = part.columns[var_col_mask][0]
    wimd = wimd.join(part.loc[:, var_col].rename(f'2014_{part_var}'))
wimd.to_file(paths.interim_data / 'wimd2014' / 'wimd2014.shp')

In [ ]:
wimd = geopd.read_file(paths.interim_data / 'wimd2014' / 'wimd2014.shp')

In [ ]:
wimd.head()

,2014_lsoa,lsoa11cd,lsoa11nm,wimd_2014,decile,map_group,2014_healt,2014_envir,2014_servi,2014_housi,2014_incom,2014_safet,2014_educa,2014_emplo,geometry
0,W01001546,W01001546,Monmouthshire 002A,403.0,3.0,30% Most Deprived,714.0,1220.0,1342.0,498.0,240.0,387.0,315.0,417.0,"POLYGON ((329425.970 214538.497, 329425.094 21..."
1,W01001545,W01001545,Monmouthshire 001A,1687.0,9.0,50% Least Deprived,1043.0,889.0,1291.0,1769.0,1599.0,1752.0,1884.0,1374.0,"POLYGON ((328804.667 214649.107, 328800.406 21..."
2,W01001542,W01001542,Monmouthshire 009A,1472.0,8.0,50% Least Deprived,1491.0,1398.0,222.0,1536.0,1434.0,1622.0,1730.0,1357.0,"POLYGON ((349359.166 191300.821, 349353.051 19..."
3,W01000088,W01000088,Gwynedd 007B,1276.0,7.0,50% Least Deprived,1461.0,1395.0,200.0,957.0,1276.0,1446.0,1288.0,1304.0,"POLYGON ((248431.557 356977.100, 248420.703 35..."
4,W01001543,W01001543,Monmouthshire 009B,1804.0,10.0,50% Least Deprived,1879.0,608.0,1353.0,1881.0,1716.0,1515.0,1573.0,1729.0,"POLYGON ((347827.977 189358.129, 347860.644 18..."


In [ ]:
metric_col = '2014_incom'

In [ ]:
wimd[metric_col].describe()

count    1909.000000
mean      955.000000
std       551.225151
min         1.000000
25%       478.000000
50%       955.000000
75%      1432.000000
max      1909.000000
Name: 2014_incom, dtype: float64

In [ ]:
wimd.plot(column=metric_col)

In [ ]:
w = Rook.from_dataframe(wimd)

In [ ]:
model = AgglomerativeClustering(
    n_clusters=None,
    connectivity=w.sparse,
    linkage="complete",
    distance_threshold=100,
    affinity='l1',
)
model.fit(wimd[[metric_col]].values)
# model = WardSpatial(wimd, w, [metric_col], n_clusters=None, clustering_kwds={'distance_threshold': 200})
# model.solve()
wimd['labels'] = model.labels_

/home/thomaslouf/.conda/envs/ses-ling/lib/python3.9/site-packages/sklearn/cluster/_agglomerative.py:549: UserWarning:

the number of connected components of the connectivity matrix is 4 > 1. Completing it to avoid stopping the tree early.



In [ ]:
clustered_wimd = wimd[['labels', 'geometry']].dissolve(by='labels')
# weighted average (by pop) instead of normal one?
clustered_wimd = clustered_wimd[['geometry']].join(wimd.groupby('labels').agg(**{k: (metric_col, k) for k in ['mean', 'min', 'max', 'size']}))

In [177]:
clustered_wimd.explore('mean')

NameError: name 'clustered_wimd' is not defined

## England

In [7]:
file_prefix = 'Indices_of_Multiple_Deprivation_(IMD)_2019'
imd = geopd.read_file(paths.ext_data / f'{file_prefix}' / f'{file_prefix}.shp').set_index('lsoa11cd')
imd.shape

(32844, 66)

In [ ]:
imd.columns

Index(['FID', 'lsoa11cd', 'lsoa11nm', 'lsoa11nmw', 'st_areasha', 'st_lengths',
       'IMD_Rank', 'IMD_Decile', 'LSOA01NM', 'LADcd', 'LADnm', 'IMDScore',
       'IMDRank0', 'IMDDec0', 'IncScore', 'IncRank', 'IncDec', 'EmpScore',
       'EmpRank', 'EmpDec', 'EduScore', 'EduRank', 'EduDec', 'HDDScore',
       'HDDRank', 'HDDDec', 'CriScore', 'CriRank', 'CriDec', 'BHSScore',
       'BHSRank', 'BHSDec', 'EnvScore', 'EnvRank', 'EnvDec', 'IDCScore',
       'IDCRank', 'IDCDec', 'IDOScore', 'IDORank', 'IDODec', 'CYPScore',
       'CYPRank', 'CYPDec', 'ASScore', 'ASRank', 'ASDec', 'GBScore', 'GBRank',
       'GBDec', 'WBScore', 'WBRank', 'WBDec', 'IndScore', 'IndRank', 'IndDec',
       'OutScore', 'OutRank', 'OutDec', 'TotPop', 'DepChi', 'Pop16_59',
       'Pop60_', 'WorkPop', 'Shape__Are', 'Shape__Len', 'geometry'],
      dtype='object')

In [8]:
metric_col = 'EduScore'

In [9]:
imd[[metric_col]].to_parquet('mdr.parquet')

In [8]:
imd[metric_col].describe()

count    32844.000000
mean        21.691084
std         18.607562
min          0.013000
25%          7.360000
50%         16.180500
75%         30.906500
max         99.446000
Name: EduScore, dtype: float64

In [ ]:
imd['TotPop'].describe()

count    32844.000000
mean      1666.307514
std        363.622458
min        523.000000
25%       1446.000000
50%       1598.000000
75%       1800.000000
max       9551.000000
Name: TotPop, dtype: float64

### Aggregate into MSOA

In [9]:
base_name = 'infuse_msoa_lyr_2011_clipped'
msoa = geopd.read_file(paths.ext_data / base_name / f'{base_name}.shp').set_index('geo_code').rename_axis('msoa_id')
print(msoa.shape)

(8480, 3)


In [277]:
lsoa_to_msoa = pd.read_csv(paths.ext_data / 'Output_Area_to_Lower_Layer_Super_Output_Area_to_Middle_Layer_Super_Output_Area_to_Local_Authority_District__December_2020__Lookup_in_England_and_Wales.csv')
lsoa_to_msoa.columns = lsoa_to_msoa.columns.str.lower()
cols_mask = lsoa_to_msoa.columns.str.match(r'(l|m)soa.+cd')
lsoa_to_msoa = lsoa_to_msoa.loc[:, cols_mask].drop_duplicates().set_index('lsoa11cd')
lsoa_to_msoa.columns = lsoa_to_msoa.columns.str.slice_replace(4, None, '_id')
lsoa_to_msoa.head()

,msoa_id
lsoa11cd,
E01011969,E02002483
E01011949,E02002491
E01011970,E02002483
E01011971,E02002483
E01011950,E02002490


In [11]:
imd = imd.join(lsoa_to_msoa, how='left')

In [12]:
grouper = imd.groupby('msoa_id')
imd['msoa_pop'] = grouper['TotPop'].transform('sum')
imd['wavg_elem'] = imd[metric_col] * imd['TotPop'] / imd['msoa_pop']
imd['avg'] = grouper[metric_col].transform('mean')
imd['wvar_elem'] = (imd[metric_col] - imd['avg'])**2 * imd['TotPop'] / imd['msoa_pop']
msoa_imd = grouper.agg(
    wavg=('wavg_elem', 'sum'),
    wvar=('wvar_elem', 'mean'),
    avg=('avg', 'first'),
    var=(metric_col, 'var'),
    min=(metric_col, 'min'),
    max=(metric_col, 'max'),
    pop=('msoa_pop', 'first'),
)
msoa_imd['wstd'] = np.sqrt(msoa_imd['wvar'])

In [13]:
msoa_imd

,wavg,wvar,avg,var,min,max,pop,wstd
msoa_id,,,,,,,,
E02000001,0.909384,10.202581,5.36350,72.967058,0.024,22.260,6687,3.194148
E02000002,7.778841,19.246909,30.48075,102.333271,17.349,38.753,7379,4.387130
E02000003,2.976738,2.308306,17.61100,17.342949,12.848,23.035,10720,1.519311
E02000004,5.729982,9.018080,23.15025,51.392207,18.702,33.858,6536,3.003012
E02000005,5.404532,8.729057,27.56860,51.124119,19.124,37.624,9243,2.954498
...,...,...,...,...,...,...,...,...
E02006930,1.583553,3.187740,6.79025,19.861708,3.229,13.311,7537,1.785424
E02006931,1.303344,3.254671,6.18140,19.642651,1.047,12.399,9405,1.804071
E02006932,5.488467,44.602976,22.97760,237.947663,4.158,43.037,12651,6.678546


In [13]:
msoa_imd['wstd'].describe()

count    6791.000000
mean        3.661148
std         2.536428
min         0.000000
25%         1.701512
50%         3.064247
75%         5.094203
max        18.475722
Name: wstd, dtype: float64

In [15]:
# checked min and max in 2015 edition of https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/middlesuperoutputareamidyearpopulationestimates
msoa_imd['pop'].describe()

count     6791.000000
mean      8058.931527
std       1772.452425
min       2335.000000
25%       6722.000000
50%       7834.000000
75%       9064.500000
max      18534.000000
Name: pop, dtype: float64

In [15]:
msoa_gdf = msoa.join(msoa_imd, how='inner')

In [58]:
simplified_mgeoms = msoa_gdf.geometry.simplify(50).to_crs('epsg:4326').rename('geometry')

In [ ]:
plot_gdf = geopd.GeoDataFrame(msoa_imd.join(simplified_mgeoms, how='inner'))

In [ ]:
m = plot_gdf.explore('wstd')

In [ ]:
m.save(paths.figs / 'imd_msoa.html')

In [17]:
msoa_gdf.shape

(6791, 11)

### Spatial aggregate by hand

In [436]:
w = Rook.from_dataframe(imd)

/home/thomaslouf/.conda/envs/ses-ling/lib/python3.9/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 9 disconnected components.
 There are 2 islands with ids: 18535, 26052.
  warnings.warn(message)


In [232]:
comp_ids, comp_inv, comp_counts = np.unique(w.component_labels, return_inverse=True, return_counts=True)
biggest_comp = comp_ids[comp_counts.argmax()]
idc_biggest_comp = np.nonzero(w.component_labels == biggest_comp)

In [82]:
full_w = w.full()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [84]:
# comp_w = full_w[idc_biggest_comp[:, np.newaxis], idc_biggest_comp]
comp_w = full_w[idc_biggest_comp, idc_biggest_comp[0]]

In [85]:
comp_w.shape

(32650, 32650)

In [174]:
from scipy import sparse
connectivity = sparse.lil_matrix(comp_w)

when two clusters are merged, need to add to the heap `inertia` and structure matrix `A` the distances not calcuated before between unconnected points. But can't edit heap, and checking every time you pop defeats the purpose of a heapq. Nah but just have to push the right distance on L243. This implies either filling A with all pairwise distances or alculating them on the fly

In [408]:
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering

rng = np.random.RandomState(0)
n_samples = 100
conn = np.ones([n_samples, n_samples], dtype=bool)
# Add some random sparsity, 70% at most
masked = (rng.random((n_samples, 70)) * n_samples).astype(int)
for i, nbh in enumerate(masked):
    conn[i, nbh] = False
    conn[nbh, i] = False
X = rng.randn(n_samples, 1)
clustering = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=1,
    connectivity=conn,
    linkage='complete',
    affinity='l1',
)
clustering.fit(X)
df = pd.DataFrame({'labels': clustering.labels_, 'x': X.flatten()})
clustered_df = df.groupby('labels').agg(**{op: ('x', op) for op in ['min', 'max']})
# This should be < distance_threshold
print((clustered_df['max'] - clustered_df['min']).max())

1.20248995732655


In [437]:
connectivity = w.sparse
idc_biggest_comp = (np.arange(imd.shape[0]), )
model = AgglomerativeClustering(
    n_clusters=None,
    connectivity=connectivity,
    linkage="complete",
    distance_threshold=5,
    affinity='l1',
)
model.fit(imd[[metric_col]].values[idc_biggest_comp])
# model = WardSpatial(wimd, w, [metric_col], n_clusters=None, clustering_kwds={'distance_threshold': 200})
# model.solve()
# imd['labels'] = model.labels_

/home/thomaslouf/.conda/envs/ses-ling/lib/python3.9/site-packages/sklearn/cluster/_agglomerative.py:549: UserWarning: the number of connected components of the connectivity matrix is 9 > 1. Completing it to avoid stopping the tree early.
  connectivity, n_connected_components = _fix_connectivity(


AgglomerativeClustering(affinity='l1',
                        connectivity=<32844x32844 sparse matrix of type '<class 'numpy.float64'>'
	with 189220 stored elements in Compressed Sparse Row format>,
                        distance_threshold=5, linkage='complete',
                        n_clusters=None)

In [446]:
comp_imd = imd.iloc[idc_biggest_comp[0]].copy()
comp_imd['labels'] = model.labels_

In [448]:
clustered_imd = comp_imd.groupby('labels').agg(**{k: (metric_col, k) for k in ['mean', 'min', 'max', 'size']})
print((clustered_imd['max'] - clustered_imd['min']).max())
print(clustered_imd.shape, comp_imd.shape)

24.389
(12198, 4) (32844, 68)


when connectivity, distance is not direct but through "path" has to take to join originally disconnected nodes, which fro complete linkage should be highes tdistance along this path

all following is consistent, so problem is actual values of distances

In [240]:
mask_pb_label = comp_imd['labels'] == clustered_imd.index[(clustered_imd['max'] - clustered_imd['min']).argmax()]
idc_pb = np.nonzero(mask_pb_label.values)

In [250]:
max_d = 0
max_i = 0
for i in idc_pb[0]:
    mask = comp_w[idc_pb[0], i].astype(bool)
    values_in_nb = comp_imd[metric_col].iloc[idc_pb[0][mask]]
    d = values_in_nb.max() - values_in_nb.min()
    if d > max_d:
        max_d = d
        max_i = i
max_d

7.823999999999999

In [252]:
mask = comp_w[idc_pb[0], max_i].astype(bool)
values_in_nb = comp_imd[metric_col].iloc[idc_pb[0][mask]]

In [256]:
comp_imd.iloc[[4075, 4085, 5099]][['labels', metric_col]]

,labels,EduScore
4075,251,14.854
4085,251,7.030
5099,251,13.722


In [253]:
values_in_nb

4075    14.854
4085     7.030
5099    13.722
Name: EduScore, dtype: float64

In [249]:
comp_imd[metric_col].iloc[idc_pb[0][mask]]

11217    1.076
11238    1.270
11851    3.356
32085    3.708
Name: EduScore, dtype: float64

In [199]:
clustered_imd.iloc[223]

mean     6.808525
min      2.884000
max     11.566000
size    59.000000
Name: 223, dtype: float64

In [450]:
clustered_imd = comp_imd[['labels', 'geometry']].dissolve(by='labels').join(clustered_imd)
# weighted average (by pop) instead of normal one?

In [452]:
map = clustered_imd.explore('mean')

In [453]:
map.save(paths.figs / 'agg_lsoa.html')

# Attribution

## Get places to cells matching

In [104]:
base_name = 'Middle_Layer_Super_Output_Areas_(December_2011)_Boundaries_Generalised_Clipped_(BGC)_EW_V3'
cells_gdf = geopd.read_file(paths.ext_data / base_name / f'{base_name}.shp').set_index('MSOA11CD').rename_axis('cell_id').rename(columns={'Shape__Are': 'area'})

In [ ]:
region_geo = cells_gdf.geometry.unary_union

In [105]:
# area_col_mask = cells_gdf.columns.str.lower().str.endswith('area')
# if area_col_mask.any():
if 'area' not in cells_gdf.columns:
    cells_gdf['area'] = cells_gdf.area
latlon_cells_gdf = cells_gdf.to_crs('epsg:4326')

In [79]:
places_filter = qr.Filter().equals('country_code', cc)
tweets_filter = qr.Filter().not_exists('coordinates.coordinates')
places_gdf = data_access.agg_places_from_mongo(year_from, year_to, places_filter, tweets_colls=colls, tweets_filter=tweets_filter)

134923it [00:12, 10594.36it/s]


In [113]:
point_places_mask = places_gdf.geom_type == 'Point'
pt_places_to_cells = places_gdf.loc[point_places_mask].sjoin(latlon_cells_gdf, predicate='intersects')['index_right'].rename(latlon_cells_gdf.index.name)

In [121]:
xy_poly_places = places_gdf.loc[~point_places_mask].to_crs(xy_proj)

In [126]:
xy_poly_places['area'] = xy_poly_places.area

Handle places from bordering countries in same collection, and those overlapping the sea

In [187]:
cells_gdf['cell_id'] = cells_gdf.index
poly_places_to_cells = xy_poly_places[['geometry', 'id', 'area']].overlay(
    cells_gdf[['geometry', 'cell_id']], how='intersection'
)
poly_places_to_cells['area_overlap'] = poly_places_to_cells.area

In [195]:
poly_places_to_cells['ratio_overlap'] = poly_places_to_cells['area_overlap'] / poly_places_to_cells['area']
poly_places_to_cells['ratio_clipped_overlap'] = poly_places_to_cells['area_overlap'] / poly_places_to_cells.groupby('id')['area_overlap'].transform('sum')

In [ ]:
poly_places_to_cells = poly_places_to_cells.set_index(['id', 'cell_id']).sort_index().drop(columns='geometry')

In [214]:
place_reg_overlap = poly_places_to_cells.groupby('id')['ratio_overlap'].sum()
idc_foreign_places = poly_places_to_cells.index.levels[0][place_reg_overlap < 0.6]
poly_places_to_cells = poly_places_to_cells.drop(index=idc_foreign_places, level='id')

In [215]:
poly_places_to_cells.head()

area  area_overlap  ratio_overlap  \
id               cell_id                                                
0001878dda20c9d3 W02000108  4.860228e+05  4.545820e+05        0.93531   
                 W02000110  4.860228e+05  3.144076e+04        0.06469   
000543740a1a65b5 E02005514  7.914599e+05  7.914599e+05        1.00000   
0008ac25d92c0023 E02006111  1.170080e+06  1.170080e+06        1.00000   
000ab548b8bc815c E02006640  5.809608e+05  5.809608e+05        1.00000   

                            ratio_clipped_overlap  
id               cell_id                           
0001878dda20c9d3 W02000108                0.93531  
                 W02000110                0.06469  
000543740a1a65b5 E02005514                1.00000  
0008ac25d92c0023 E02006111                1.00000  
000ab548b8bc815c E02006640                1.00000

In [252]:
# since groubpy preserves order within each group, the cumsum is computed from the
# largest to the smallest value in each group. Then we take the first four (at most, if
# there are fewer rows they are kept, unlike when using `nth`), in order to take the
# fourth. For each place, this value then corresponds to the total ratio of overlap of
# the place summed over the 4 cells it overlaps the most with.
ntop = 2
places_top_cells_cumoverlap = poly_places_to_cells.sort_values(by='ratio_overlap', ascending=False).groupby('id').cumsum().groupby('id')['ratio_clipped_overlap'].head(ntop).groupby('id').last()
places_to_remove = places_top_cells_cumoverlap.index[places_top_cells_cumoverlap < 0.9]

In [249]:
places_gdf.loc[places_top_cells_cumoverlap.index].loc[places_top_cells_cumoverlap < 0.9, 'nr_tweets'].sum() * 1e-6

31.886827999999998

In [268]:
places_to_cells = pd.concat([
    pt_places_to_cells.to_frame().assign(ratio=1).set_index('cell_id', append=True),
    poly_places_to_cells['ratio_clipped_overlap'].rename('ratio').drop(index=places_to_remove, level='id').to_frame()
]).sort_index()

In [270]:
places_to_cells.head()

ratio
id               cell_id           
0001878dda20c9d3 W02000108  0.93531
                 W02000110  0.06469
000543740a1a65b5 E02005514  1.00000
0008ac25d92c0023 E02006111  1.00000
000ab548b8bc815c E02006640  1.00000

for each user: 
 * for bots, get first and last tweet time,
 * for each place, count how many times tweeted at day/night time
 * for coordinates, do it apart? by df chunk of tweets into sjoin? this way you then aggregate by cell_id by time of day
 
then treat user/place to go to user/cell, append gps data and finally do the attribution:
have user / cell / is_daytime | count (float)
take at nighttime rel frequencies of presence in cells: if entropy not too high, then take the one with highest frequency, or cell in which geometric median falls (pb: agg then pt then agg again)

dict with for every year the ones to discard? or only global

In [561]:
latlon_cells_gdf = cells_gdf.to_crs('epsg:4326')

## Load user counts per cell and place

In [566]:
save_path = paths.resident_ids
with open(str(save_path).format(cc=cc, year_from=2015, year_to=2021), 'r') as f:
    resident_ids = f.read().splitlines()

In [596]:
len(resident_ids)

1505107

In [273]:
save_path = str(paths.user_cells_from_gps).format(cell_kind='LSOA_BGC')
lsoa_user_counts = pd.read_parquet(save_path)

In [344]:
lsoa_user_counts.head()

count
user_id                                            cell_id   is_daytime       
000003962867f2d312d942838cdb2535b950589f2e85f31... E01002819 False         2.0
                                                             True          1.0
                                                   E01002821 False         2.0
                                                   E01002828 True          1.0
                                                   E01002862 False         1.0

In [404]:
lsoa_to_msoa.head()

,msoa_id
lsoa11cd,
E01011969,E02002483
E01011949,E02002491
E01011970,E02002483
E01011971,E02002483
E01011950,E02002490


In [364]:
user_gps_counts = lsoa_user_counts.join(lsoa_to_msoa.rename_axis('cell_id')).groupby(['user_id', 'msoa_id', 'is_daytime']).sum().rename_axis(index={'msoa_id': 'cell_id'})

In [308]:
user_gps_counts.head()

count
user_id                                            cell_id   is_daytime       
000003962867f2d312d942838cdb2535b950589f2e85f31... E02000588 False         2.0
                                                             True          1.0
                                                   E02000590 False         2.0
                                                   E02000584 True          1.0
                                                   E02000590 False         1.0

In [311]:
save_path = str(paths.user_places)
user_places_counts = pd.read_parquet(save_path)

In [316]:
user_places_counts.head()

count
user_id                                            place_id         is_daytime       
000003962867f2d312d942838cdb2535b950589f2e85f31... 01c7ed8caf11e8b2 False         1.0
                                                                    True          1.0
                                                   151b9e91272233d1 True          1.0
                                                   315b740b108481f6 False       245.0
                                                                    True        136.0

In [374]:
user_counts = user_places_counts.join(places_to_cells.reset_index().set_index('id'), on='place_id', how='inner').groupby(['user_id', 'cell_id', 'is_daytime']).sum()

In [375]:
user_counts.head()

count  \
user_id                                            cell_id   is_daytime          
000018e3a47e911eae3f9a2be451be87fefa3755b0f8b5b... E02003654 False         2.0   
                                                             True          6.0   
                                                   E02003656 False         2.0   
                                                             True          6.0   
                                                   E02006523 True          2.0   

                                                                            ratio  
user_id                                            cell_id   is_daytime            
000018e3a47e911eae3f9a2be451be87fefa3755b0f8b5b... E02003654 False       0.025876  
                                                             True        0.025876  
                                                   E02003656 False       0.974124  
                                                             True        0.974124  
                                                   E02006523 True        1.000000

In [327]:
user_counts.index.levels[0].size, user_places_counts.index.levels[0].size

(779839, 1407022)

In [376]:
user_counts['count'] = user_counts['count'] * user_counts['ratio']
user_counts = user_counts.add(user_gps_counts, fill_value=0)

In [371]:
user_counts.index.levels[0].size

1078550

In [378]:
user_counts['prop_user'] = user_counts['count'] / user_counts.groupby('user_id')['count'].transform('sum')
user_counts['prop_user_by_time'] = user_counts['count'] / user_counts.groupby(['user_id', 'is_daytime'])['count'].transform('sum')

In [379]:
user_counts.head()

count  \
user_id                                            cell_id   is_daytime          
000003962867f2d312d942838cdb2535b950589f2e85f31... E02000584 True          1.0   
                                                   E02000586 True          1.0   
                                                   E02000588 False         2.0   
                                                             True          1.0   
                                                   E02000590 False         3.0   

                                                                         ratio  \
user_id                                            cell_id   is_daytime          
000003962867f2d312d942838cdb2535b950589f2e85f31... E02000584 True          NaN   
                                                   E02000586 True          NaN   
                                                   E02000588 False         NaN   
                                                             True          NaN   
                                                   E02000590 False         NaN   

                                                                         prop_user  \
user_id                                            cell_id   is_daytime              
000003962867f2d312d942838cdb2535b950589f2e85f31... E02000584 True         0.014085   
                                                   E02000586 True         0.014085   
                                                   E02000588 False        0.028169   
                                                             True         0.014085   
                                                   E02000590 False        0.042254   

                                                                         prop_user_by_time  
user_id                                            cell_id   is_daytime                     
000003962867f2d312d942838cdb2535b950589f2e85f31... E02000584 True                 0.027778  
                                                   E02000586 True                 0.027778  
                                                   E02000588 False                0.057143  
                                                             True                 0.027778  
                                                   E02000590 False                0.085714

In [426]:
user_residence_cells = user_counts.loc[(user_counts['prop_user_by_time'] > 0.5) & (user_counts['prop_user'] > 0.1)].loc[(slice(None), slice(None), False)]

In [434]:
assign_kwargs = dict(
    nighttime_acty_th = 0.5,
    all_acty_th = 0.1,
    count_th = 3,
)
user_residence_cells = user_residence.assign(user_counts, **assign_kwargs)

In [451]:
paths.partial_format(**assign_kwargs)
user_residence_cells.to_parquet(paths.user_residence_cell, index=True)

In [ ]:
cells_gdf[['geometry']].join(user_residence_cells.groupby('cell_id').size().rename('count')).fillna(0).explore('count')

In [465]:
save_path = str(paths.user_cells_from_gps).format(cell_kind='LSOA_BGC')
a = pd.read_parquet(save_path).groupby('user_id').sum()
save_path = str(paths.user_places)
a = a.add(
    pd.read_parquet(save_path).groupby('user_id').sum(),
    fill_value=0
)

In [546]:
save_path = str(paths.user_cells_from_gps).format(cell_kind='LSOA_BGC')
a = pd.read_parquet(save_path).groupby('user_id').sum()

In [476]:
x = a.loc[user_residence_cells.index, 'count'].sort_values(ascending=False)

In [478]:
cumsum = x.cumsum()
idx_split = (cumsum > 0.5 * cumsum[-1]).argmax()
max_chunk = 5e6
list_chunks = []
reverse_cumsum = cumsum[:idx_split:-1]

In [477]:
cumsum = x.cumsum()
idx_split = (cumsum > 0.5 * cumsum[-1]).argmax()
max_chunk = 5e6
list_chunks = []
reverse_cumsum = cumsum[-1] - cumsum[:idx_split:-1]
start_idx = 0
for i in range(idx_split):
    list_chunks.append(i)
    stop_idx = start_idx + (reverse_cumsum[start_idx:] - reverse_cumsum[start_idx:][0] > max_chunk).argmax()
    
    start_idx = stop_idx 

14233

In [509]:
cumsum = x.cumsum()
idx_split = (cumsum > 0.5 * cumsum[-1]).argmax()
max_chunk = 5e6
list_chunks = []
reverse_cumsum = cumsum[-1] - cumsum[:idx_split-1:-1]
user_chunks = pd.concat([
    cumsum[:idx_split] // (max_chunk / 2),
    reverse_cumsum // (max_chunk / 2)
]).astype(int).sort_values()
nr_users = x.shape[0]
obj_chunk_size = nr_users // user_chunks.values[-1]

In [515]:
user_chunk_df = pd.concat([x, user_chunks.rename('chunk')], axis=1)
chunks_df = pd.concat(
    [
        user_chunk_df.groupby('chunk').size().rename('nr_users'),
        user_chunk_df.groupby('chunk')['count'].sum().rename('nr_tweets')
    ],
    axis=1,
)
    

In [538]:
user_chunk_df['rand_chunk'] = (np.random.random(nr_users) * (cumsum[-1] // max_chunk)).astype(int)

In [539]:
rand_chunks_df = pd.concat(
    [
        user_chunk_df.groupby('rand_chunk').size().rename('nr_users'),
        user_chunk_df.groupby('rand_chunk')['count'].sum().rename('nr_tweets')
    ],
    axis=1,
)

In [543]:
user_chunk_df['rand_chunk'] = (np.random.random(nr_users) * (cumsum[-1] // max_chunk)).astype(int)
rand_chunks_df = pd.concat(
    [
        user_chunk_df.groupby('rand_chunk').size().rename('nr_users'),
        user_chunk_df.groupby('rand_chunk')['count'].sum().rename('nr_tweets')
    ],
    axis=1,
)
std = rand_chunks_df['nr_tweets'].std()
while std > 0.1 * max_chunk:
    user_chunk_df['rand_chunk'] = (np.random.random(nr_users) * (cumsum[-1] // max_chunk)).astype(int)
    rand_chunks_df = pd.concat(
        [
            user_chunk_df.groupby('rand_chunk').size().rename('nr_users'),
            user_chunk_df.groupby('rand_chunk')['count'].sum().rename('nr_tweets')
        ],
        axis=1,
    )
    std = rand_chunks_df['nr_tweets'].std()

201974.35186885946